# TD Méthodes d'apprentissage

Merci de renseigner votre groupe:

* PRÉNOM NOM
* PRÉNOM NOM

# Prédiction d'AVC

Vous trouverez dans cette archive un jeu de données (`healthcare-dataset-stroke-data.csv` -
[source](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)) portant sur des patients suspecté
d'avoir eu un AVC.

Les colonnes suivantes sont renseignées:

* `gender`: le genre (`Male`, `Female` ou `Other`) du patient,
* `age`: l'age du patient,
* `hypertension`: 0 si le patient n'a pas d'hypertension, 1 sinon,
* `hear_disease`: 0 si le patient n'a jamais eu de maladie cardiaque, 1 sinon,
* `ever_married`: `No` ou `Yes` si le patient a déjà été marié,
* `work_type`: l'emploi du patient (`children`, `Govt_job`, `Never_worked`, `Private` or `Self-employed`),
* `Residence_type`: `rural` ou `urbain`,
* `avg_glucose_level`: le niveau de glucose dans le sang,
* `bmi`: l'IMC (*body mass index*),
* `smoking_status`: selon si le patient fume: `formerly smoked`, `never smoked` `smokes` or `Unknown`,
* `stroke`: 1 si le patient a un AVC, 0 sinon (la variable que l'on souhaite prédire).

## 1. Chargement des données

### 1.1 Chargez les données:

In [7]:
import pandas

# À faire

### 1.2 Le nombre de lignes qu'elles contiennent:

In [8]:
# À faire

### 1.3 Le % de personnes qui ont eu un AVC

In [9]:
# À faire

### 1.4 Le % d'hommes et le % de femmes qui ont eu un AVC

In [10]:
# À faire

### 1.5 Un histogramme des ages, pour les personnes ayant eu un AVC et celles n'ayant pas eu d'AVC:

In [11]:
# À faire

## 2. Moindres carrés

### 2.1 `age` vs `bmi`

Tracez le graphique (en nuage de points) ayant en abscisse l'age et en ordonnées l'indice de masse corporel (`bmi`):

In [13]:
# À faire

### 2.2 Modèle linéaire

À partir de la courbe ci-dessus, trouvez les paramètres $f(x) = ax + b$ optimums au sens des moindres carrés

In [14]:
# À faire

Tracez à nouveau le nuage de point, en superposant la courbe trouvée

In [15]:
# À faire

Que pensez vous du résultat ?

*A COMPLETER*

### 2.3 Modèle quadratique

Faites la même chose, mais pour le modèle $f(x) = ax^2 + bx + c$

In [16]:
# À faire

Vous devriez constater trois "phases": l'IMC monte (avec la croissance), puis stagne, et enfin baisse. 

À partir de votre modèle, déterminez à quel age l'IMC cesse de monter et commence à baisser.

*A COMPLETER*

## 3. Prédiction des AVC



### 3.1 Préparation des données

Nous allons préparer les données, notamment:

* Enlever la colonne  `id`, qui ne nous servira pas à l'entraînement,
* À l'aide de la méthode [get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html), les colonnes prenant des valeurs discrètes seront remplacées par leur représentation [one hot](https://en.wikipedia.org/wiki/One-hot)
  * *Par exemple, au lieu de la colonne `gender` qui pourra valoir `Male` ou `Female`, on aura `gender_Male` et `gender_Female` qui pourront valoir 0 ou 1.*

In [30]:
# À faire

### 3.1 Séparation des données d'entraînement / test

Maintenant, nous allons séparer ces données en:

* Un jeu d'entraînement (90%),
* Un jeu de test (10%).

[indication](https://stackoverflow.com/a/35531218/3776931)

In [17]:
# À faire

### 3.3 Chargement de `torch`

On chargera `torch`, et on déclarera un perceptron multi-couche avec deux couches cachées:

In [27]:
import torch as th


class MLP(th.nn.Module):
    def __init__(self, input_dimension: int):
        super().__init__()

        layers = [
            th.nn.Linear(input_dimension, 128),
            th.nn.ReLU(),
            th.nn.Linear(128, 128),
            th.nn.ReLU(),
            th.nn.Linear(128, 1),
            th.nn.Sigmoid()
        ]

        self.net = th.nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


Notez que la sortie de ce réseau est une fonction d'activation (sigmoïde) qui produira une valeur entre 0 et 1.

Créez maintenant votre réseau en définissant son nombre d'entrées:

In [29]:
# net = MLP(nb_valeurs)

### 3.4: Échantillonage des données

Écrivez une fonction qui sélectionne au hasard N lignes des données d'entraînement:



In [33]:
def sample_data(n=128):

    # TODO: extraire 128 lignes au hasard des données d'entraînement
    # - xs devrait être de taille n x nb_valeurs
    # - ys devrait être de taille n
    
    return xs, ys

Vous pourrez compléter le code suivant afin d'entraîner votre réseau. Vous utiliserez la perte [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html):

In [ ]:
# Création d'un optimiseur
optimizer = th.optim.Adam(net.parameters(), 1e-4)

# Pour 1024 étapes d'entraînement
for k in range(1024):
    # On échantillonne 128 données
    data = sample_data(128)

    # Fonction de perte
    loss = ...

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

### 3.5: Évaluation de votre réseau

En utilisant vos données de test, mesurez le % des données qui est bien étiquetée par le réseau

In [34]:
# À faire

Calculez maintenant le % de données positives (stroke=1) et négatives (stroke=0) bien étiquetées

In [35]:
# À faire

Calculez les nombres:
* **TP**: Vrais positifs
* **TN**: Vrais négatifs
* **FP**: Faux positifs
* **FN**: Faux négativs

In [37]:
# À faire

### 4. Quelques métriques

On définit l'exactitude comme le % de données bien étiquetées (indépendament du fait qu'elles soient positives ou négatives).

Calculez l'exactitude de votre prédiction:

In [1]:
# À faire

Réécrivez votre boucle d'entraînement afin de produire un graphique montrant, pour chaque étape de l'entraînement:

- La loss sur le jeu de test,
- La loss sur le jeu d'entraînement,
- L'exactitude sur le jeu de test,
- L'exactitude sur le jeu d'entraînement.

In [2]:
# À faire